In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

import torch
import scgen
from feature_attribution_sc.explainers.mask import mask, generate_rankings
from feature_attribution_sc.models.scgen_models import SCGENCustom

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

Global seed set to 0
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/numba/core/cpu.py:78: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ModuleNotFoundError(No module named 'numba.targets')'.
  numba.core.entrypoints.init_all()
2023-03-13 18:55:53.040727: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-13 18:55:53.040892: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-13 18:55:53.040897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing librari

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.10.0 pandas==1.5.0 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.10.3 pynndescent==0.5.7


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
data_path='/home/yuge/feature-attribution-sc'

feature_importance_files = [
#     f'{data_path}/outputs/baselines/task1_random.csv',
    f'{data_path}/outputs/differential_expression/task1_DE_control.csv',  # these are way slower for some reason???
    f'{data_path}/outputs/expected_gradients/task1_absolute_expected_grads_v2.csv'
]

thresholds = list(range(0, 100, 10))

In [5]:
adata = sc.read(f'{data_path}/datasets/2301_scgen_norman19.h5ad')

### Train new models

Train a new scGen model per attribution dataframe and per threshold representing the percentage of data points ablated. Significantly decreased performance _after_ training indicates the data points ablated were significant/unable to be replaced by correlated variables per Hooker 2019.

In [ ]:
for feature_importance in feature_importance_files:
    attrib_df = pd.read_csv(feature_importance)
    attrib_key = feature_importance.split("/")[-2]

    # reverse order for DE
    if 'differential_expression' in feature_importance:
        attrib_df[attrib_df.columns[1:]] *= -1

    # todo: should include a check for adata.var_names vs attrib_df.gene_symbols

    n_features = attrib_df.shape[0]
    if n_features < adata.shape[1]:
        raise ValueError(
            f"Attributions only calculated for {n_features} genes but adata has {adata.shape[1]}")
    elif n_features > adata.shape[1]:
        print(f'Only using attributions for {adata.shape[1]} genes.')
        attrib_df = attrib_df.set_index('gene_symbols').loc[adata.var_names].reset_index()

    for threshold in thresholds:
        try:
            print('training at threshold =', threshold)
            SCGENCustom.setup_anndata(adata)
            save_str = f'../models/scgen_norman19_ROAR_{attrib_key}_{threshold}'
            model = SCGENCustom(adata, feature_importance=attrib_df, threshold=threshold, n_hidden=400, n_latent=30)
            model.train(max_epochs=50, batch_size=32)
            model.save(save_str, overwrite=True)

            # plot performance
            pd.concat(model.history.values(), axis=1).plot()
            plt.legend(bbox_to_anchor=(1.01, 1.05))
            plt.show()
        except:
            continue

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Only using attributions for 880 genes.
training at threshold = 0
Generating rankings for 107 labels and 880 features.
Epoch 37/50:  72%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                   | 36/50 [12:23<05:07, 21.98s/it, loss=38.8, v_num=1]